# M12 Seminar: Economic and Social Problems: Insights from Big Data.
# Term Paper. 
# Replication file №4 - out-of-sample analysis (prediction) + XGBoost extension

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

### Load US time series data
### Files can be found [here](https://github.com/social-connectedness-index/example-scripts)

In [2]:
ts_df = pd.read_csv('covid19_exploration/_intermediate/time_series_regress_dat.csv', encoding='cp1251')
ts_df['date'] = pd.to_datetime(ts_df['date'])
print(ts_df.shape)

(53363, 91)


In [3]:
# unique counties
counties = pd.unique(ts_df['county_fips'])
print(len(counties))

3139


### Modelling

In [4]:
# target variable
target = ['log_chg_cases_10k']
# baseline features
baseline_cases = ['log_l1_chg_dwc_10k', 
                  'log_l2_chg_dwc_10k', 
                  'log_l1_chg_cases_10k',
                  'log_l2_chg_cases_10k',
                  'med_hhinc2016_10k', 
                  'popdensity2010_10k']
# lex features
lex_cases_vars = ['log_l1_chg_lwc_10k', 'log_l2_chg_lwc_10k']
# sci features
sci_cases_vars = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k']
# goggle features
google_vars = ['l1_pchg_gogl_fever', 'l1_pchg_gogl_cough', 'l1_pchg_gogl_fatigue']

In [1]:
# function that performs out-of-sample forecast (random forest)
# returns RMSE on train and test (out-of-sample) data
def fit_random_forest(x_train, y_train, x_test, y_test):
    rfr = RandomForestRegressor(n_estimators=500, max_depth=3, random_state=777, n_jobs=-1)
    rfr.fit(x_train, y_train)
    rfr_test_pred = rfr.predict(x_test)
    rfr_train_pred = rfr.predict(x_train)
    train_loss = mean_squared_error(y_true=y_train, y_pred=rfr_train_pred) ** 0.5
    test_loss = mean_squared_error(y_true=y_test, y_pred=rfr_test_pred) ** 0.5
    return train_loss, test_loss

In [29]:
# function that performs out-of-sample forecast (gradient boosting)
# returns RMSE on train and test (out-of-sample) data
def fit_xgboost(x_train, y_train, x_test, y_test, max_depth=3):
    xgb = xgboost.XGBRegressor(n_estimators=500, max_depth=max_depth, random_state=777, n_jobs=-1)
    xgb.fit(x_train, y_train)
    xgb_test_pred = xgb.predict(x_test)
    xgb_train_pred = xgb.predict(x_train)
    train_loss = mean_squared_error(y_true=y_train, y_pred=xgb_train_pred) ** 0.5
    test_loss = mean_squared_error(y_true=y_test, y_pred=xgb_test_pred) ** 0.5
    return train_loss, test_loss

In [35]:
# supplementary function for model training
def get_train_test_data(train, test, colnames):
    x_train = train[colnames + target].dropna()
    y_train = x_train[target]
    x_train = x_train.drop(target, axis=1)
    x_test = test[colnames].fillna(0)
    y_test = test[target]
    return x_train, y_train, x_test, y_test

In [9]:
# iterative out-of-sample forecasting (random forest and gradient boosting)
rfr_baseline_losses = []
rfr_baseline_sci_losses = []

xgb_baseline_losses = []
xgb_baseline_sci_losses = []

rfr_baseline_lex_google_losses = []
rfr_baseline_lex_google_sci_losses = []

xgb_baseline_lex_google_losses = []
xgb_baseline_lex_google_sci_losses = []

for week_num in tqdm(range(17, 45, 2)):
    train = ts_df.loc[ts_df['week_num'] < week_num]
    test = ts_df.loc[ts_df['week_num'] == week_num]
    
    x_train_baseline, y_train_baseline, x_test_baseline, y_test_baseline = get_train_test_data(train, test, baseline_cases)
    print(f'Baseline train DF shape : {x_train_baseline.shape}')
    
    x_train_baseline_sci, y_train_baseline_sci, x_test_baseline_sci, y_test_baseline_sci = get_train_test_data(train, test, baseline_cases + sci_cases_vars)
    print(f'Baseline+SCI train DF shape : {x_train_baseline_sci.shape}')
    
    x_train_baseline_lex_google, y_train_baseline_lex_google, x_test_baseline_lex_google, y_test_baseline_lex_google = get_train_test_data(train, test, baseline_cases + lex_cases_vars + google_vars)
    print(f'Baseline+LEX+Google train DF shape : {x_train_baseline_lex_google.shape}')
    
    x_train_baseline_lex_google_sci, y_train_baseline_lex_google_sci, x_test_baseline_lex_google_sci, y_test_baseline_lex_google_sci = get_train_test_data(train, test, baseline_cases + lex_cases_vars + google_vars + sci_cases_vars)
    print(f'Baseline+LEX+Google+SCI train DF shape : {x_train_baseline_lex_google_sci.shape}')
    
    
    # baseline random forest
    rfr_train_loss, rfr_test_loss = fit_random_forest(x_train=x_train_baseline, 
                                                      y_train=y_train_baseline, 
                                                      x_test=x_test_baseline, 
                                                      y_test=y_test_baseline)
    
    rfr_baseline_losses.append([rfr_train_loss, rfr_test_loss])
    
    # baseline xgboost
    xgb_train_loss, xgb_test_loss = fit_xgboost(x_train=x_train_baseline, 
                                                y_train=y_train_baseline, 
                                                x_test=x_test_baseline, 
                                                y_test=y_test_baseline)
    
    xgb_baseline_losses.append([xgb_train_loss, xgb_test_loss])
    
    # baseline + sci random forest
    rfr_train_loss, rfr_test_loss = fit_random_forest(x_train=x_train_baseline_sci, 
                                                      y_train=y_train_baseline_sci, 
                                                      x_test=x_test_baseline_sci, 
                                                      y_test=y_test_baseline_sci)
    
    rfr_baseline_sci_losses.append([rfr_train_loss, rfr_test_loss])
    
    # baseline + sci xgboost
    xgb_train_loss, xgb_test_loss = fit_xgboost(x_train=x_train_baseline_sci, 
                                                      y_train=y_train_baseline_sci, 
                                                      x_test=x_test_baseline_sci, 
                                                      y_test=y_test_baseline_sci)
    
    xgb_baseline_sci_losses.append([xgb_train_loss, xgb_test_loss])
    
    # baseline + LEX + Google random forest
    rfr_train_loss, rfr_test_loss = fit_random_forest(x_train=x_train_baseline_lex_google, 
                                                      y_train=y_train_baseline_lex_google, 
                                                      x_test=x_test_baseline_lex_google, 
                                                      y_test=y_test_baseline_lex_google)
    
    rfr_baseline_lex_google_losses.append([rfr_train_loss, rfr_test_loss])
    
    # baseline + LEX + Google xgboost
    xgb_train_loss, xgb_test_loss = fit_xgboost(x_train=x_train_baseline_lex_google, 
                                                      y_train=y_train_baseline_lex_google, 
                                                      x_test=x_test_baseline_lex_google, 
                                                      y_test=y_test_baseline_lex_google)
    
    xgb_baseline_lex_google_losses.append([xgb_train_loss, xgb_test_loss])
    
    # baseline + LEX + Google + sci random forest
    rfr_train_loss, rfr_test_loss = fit_random_forest(x_train=x_train_baseline_lex_google_sci, 
                                                      y_train=y_train_baseline_lex_google_sci, 
                                                      x_test=x_test_baseline_lex_google_sci, 
                                                      y_test=y_test_baseline_lex_google_sci)
    
    rfr_baseline_lex_google_sci_losses.append([rfr_train_loss, rfr_test_loss])
    
    # baseline + LEX + Google + sci xgboost
    xgb_train_loss, xgb_test_loss = fit_xgboost(x_train=x_train_baseline_lex_google_sci, 
                                                      y_train=y_train_baseline_lex_google_sci, 
                                                      x_test=x_test_baseline_lex_google_sci, 
                                                      y_test=y_test_baseline_lex_google_sci)
    
    xgb_baseline_lex_google_sci_losses.append([xgb_train_loss, xgb_test_loss])
    print()

Baseline train DF shape : (3139, 6)
Baseline+SCI train DF shape : (3139, 8)
Baseline+LEX+Google train DF shape : (1976, 11)
Baseline+LEX+Google+SCI train DF shape : (1976, 13)

Baseline train DF shape : (6278, 6)
Baseline+SCI train DF shape : (6278, 8)
Baseline+LEX+Google train DF shape : (3952, 11)
Baseline+LEX+Google+SCI train DF shape : (3952, 13)

Baseline train DF shape : (9417, 6)
Baseline+SCI train DF shape : (9417, 8)
Baseline+LEX+Google train DF shape : (5928, 11)
Baseline+LEX+Google+SCI train DF shape : (5928, 13)

Baseline train DF shape : (12556, 6)
Baseline+SCI train DF shape : (12556, 8)
Baseline+LEX+Google train DF shape : (7904, 11)
Baseline+LEX+Google+SCI train DF shape : (7904, 13)

Baseline train DF shape : (15695, 6)
Baseline+SCI train DF shape : (15695, 8)
Baseline+LEX+Google train DF shape : (9880, 11)
Baseline+LEX+Google+SCI train DF shape : (9880, 13)

Baseline train DF shape : (18834, 6)
Baseline+SCI train DF shape : (18834, 8)
Baseline+LEX+Google train DF shap

In [11]:
# transfer train and test RMSEs into numpy.ndarray format
rfr_baseline_losses = np.array(rfr_baseline_losses)
rfr_baseline_sci_losses = np.array(rfr_baseline_sci_losses)

xgb_baseline_losses = np.array(xgb_baseline_losses)
xgb_baseline_sci_losses = np.array(xgb_baseline_sci_losses)

rfr_baseline_lex_google_losses = np.array(rfr_baseline_lex_google_losses)
rfr_baseline_lex_google_sci_losses = np.array(rfr_baseline_lex_google_sci_losses)

xgb_baseline_lex_google_losses = np.array(xgb_baseline_lex_google_losses)
xgb_baseline_lex_google_sci_losses = np.array(xgb_baseline_lex_google_sci_losses)

In [12]:
# construct DataFrame with RMSEs of all models
replication_df = pd.DataFrame()

replication_df['rfr_baseline_losses_train'] = rfr_baseline_losses[:, 0]
replication_df['rfr_baseline_losses_test'] = rfr_baseline_losses[:, 1]

replication_df['rfr_baseline_sci_losses_train'] = rfr_baseline_sci_losses[:, 0]
replication_df['rfr_baseline_sci_losses_test'] = rfr_baseline_sci_losses[:, 1]

replication_df['xgb_baseline_losses_train'] = xgb_baseline_losses[:, 0]
replication_df['xgb_baseline_losses_test'] = xgb_baseline_losses[:, 1]

replication_df['xgb_baseline_sci_losses_train'] = xgb_baseline_sci_losses[:, 0]
replication_df['xgb_baseline_sci_losses_test'] = xgb_baseline_sci_losses[:, 1]

replication_df['rfr_baseline_lex_google_losses_train'] = rfr_baseline_lex_google_losses[:, 0]
replication_df['rfr_baseline_lex_google_losses_test'] = rfr_baseline_lex_google_losses[:, 1]

replication_df['rfr_baseline_lex_google_sci_losses_train'] = rfr_baseline_lex_google_sci_losses[:, 0]
replication_df['rfr_baseline_lex_google_sci_losses_test'] = rfr_baseline_lex_google_sci_losses[:, 1]

replication_df['xgb_baseline_lex_google_losses_train'] = xgb_baseline_lex_google_losses[:, 0]
replication_df['xgb_baseline_lex_google_losses_test'] = xgb_baseline_lex_google_losses[:, 1]

replication_df['xgb_baseline_lex_google_sci_losses_train'] = xgb_baseline_lex_google_sci_losses[:, 0]
replication_df['xgb_baseline_lex_google_sci_losses_test'] = xgb_baseline_lex_google_sci_losses[:, 1]

In [13]:
# save DataFrame in .csv format
replication_df.to_csv('replication_oos_rmse_rf_xgb.csv', index=False)

In [14]:
replication_df

,rfr_baseline_losses_train,rfr_baseline_losses_test,rfr_baseline_sci_losses_train,rfr_baseline_sci_losses_test,xgb_baseline_losses_train,xgb_baseline_losses_test,xgb_baseline_sci_losses_train,xgb_baseline_sci_losses_test,rfr_baseline_lex_google_losses_train,rfr_baseline_lex_google_losses_test,rfr_baseline_lex_google_sci_losses_train,rfr_baseline_lex_google_sci_losses_test,xgb_baseline_lex_google_losses_train,xgb_baseline_lex_google_losses_test,xgb_baseline_lex_google_sci_losses_train,xgb_baseline_lex_google_sci_losses_test
0,0.669652,1.124411,0.651928,1.330899,0.298636,1.701912,0.264178,1.620042,0.548788,1.220528,0.543343,1.326225,0.124243,1.158518,0.106177,1.282564
1,0.755244,0.775609,0.754838,0.775406,0.440828,0.983207,0.404515,0.961906,0.662272,0.865908,0.662111,0.849453,0.249522,1.121128,0.237589,1.207949
2,0.762738,0.741119,0.762508,0.741030,0.507057,0.830562,0.477784,0.775607,0.666381,0.829693,0.665967,0.805936,0.328833,1.025142,0.310682,1.156474
3,0.757707,0.723681,0.757696,0.723586,0.540714,0.737487,0.513552,0.715478,0.657579,0.764084,0.657157,0.756091,0.370802,0.958089,0.363219,1.172175
4,0.753182,0.809167,0.752931,0.807932,0.562597,0.817999,0.536566,0.799842,0.653659,0.831459,0.653390,0.823574,0.398549,0.929068,0.385329,0.932373
5,0.764238,0.869110,0.762991,0.854253,0.592883,0.881255,0.566314,0.869327,0.663504,0.843810,0.663394,0.845505,0.427720,0.877050,0.417948,0.859528
6,0.780744,0.881134,0.774764,0.861464,0.616987,0.893641,0.591017,0.922350,0.669174,0.821221,0.668374,0.820134,0.438885,0.903794,0.430292,0.822174
7,0.788833,0.694712,0.778617,0.675934,0.622944,0.810184,0.599696,0.829277,0.664334,0.709299,0.663530,0.698256,0.442985,0.837649,0.435387,0.851547
8,0.779220,0.711245,0.770295,0.722962,0.627419,0.827886,0.604393,0.806893,0.650919,0.719212,0.650074,0.735772,0.445284,1.025254,0.439898,1.039725
9,0.774038,0.760590,0.765941,0.758032,0.627335,0.767380,0.606709,0.760211,0.637234,0.769444,0.637021,0.770078,0.443288,0.870846,0.439217,0.867469


### Extension - XGBoost model on all available data (lagged values and controls)

In [62]:
# select lagged variables
cols_extended = []
cols_sci = []
for colname in ts_df.columns:
    if 'l1' in colname or 'l2' in colname:
        if 'swc' in colname or 'swd' in colname:
            cols_sci.append(colname)
        else:
            cols_extended.append(colname)
# add fixed control variables
cols_extended += ['med_hhinc2016_10k', 
                  'popdensity2010_10k',  
                  'share_within50', 
                  'share_within150']

In [63]:
# list af all features used
cols_extended

['l1_pchg_gogl_fever',
 'l1_pchg_gogl_cough',
 'l1_pchg_gogl_fatigue',
 'l1_chg_dwc_10k',
 'l2_chg_dwc_10k',
 'l1_chg_lwc_10k',
 'l2_chg_lwc_10k',
 'l1_chg_cases_10k',
 'l2_chg_cases_10k',
 'l2_chg_dwd_10k_4wk',
 'l2_chg_lwd_10k_4wk',
 'l2_chg_deaths_10k_4wk',
 'log_l1_chg_cases_10k',
 'log_l2_chg_cases_10k',
 'log_l1_chg_dwc_10k',
 'log_l2_chg_dwc_10k',
 'log_l1_chg_lwc_10k',
 'log_l2_chg_lwc_10k',
 'log_l2_chg_deaths_10k_4wk',
 'log_l2_chg_dwd_10k_4wk',
 'log_l2_chg_lwd_10k_4wk',
 'med_hhinc2016_10k',
 'popdensity2010_10k',
 'share_within50',
 'share_within150']

In [64]:
# list of SCI features
cols_sci

['l1_chg_swc_10k',
 'l2_chg_swc_10k',
 'l2_chg_swd_10k_4wk',
 'log_l1_chg_swc_10k',
 'log_l2_chg_swc_10k',
 'log_l2_chg_swd_10k_4wk']

In [65]:
# supplementary data management function
def get_train_test_data(train, test, colnames):
    x_train = train[colnames + target]
    x_train = x_train.dropna()
    y_train = x_train[target]
    x_train = x_train.drop(target, axis=1)
    x_test = test[colnames].fillna(0)
    y_test = test[target]
    return x_train, y_train, x_test, y_test

In [77]:
# iterative out-of-sample prediction
ext_losses = []
ext_sci_losses = []

for week_num in tqdm(range(17, 45, 2)):
    train = ts_df.loc[ts_df['week_num'] < week_num]
    test = ts_df.loc[ts_df['week_num'] == week_num]
    
    x_train, y_train, x_test, y_test = get_train_test_data(train, test, cols_extended)
    print(f'Baseline train DF shape : {x_train.shape}')
    
    x_train_sci, y_train_sci, x_test_sci, y_test_sci = get_train_test_data(train, test, cols_extended + cols_sci)
    print(f'Baseline+SCI train DF shape : {x_train_sci.shape}')
    
    # xgboost without SCI
    train_loss, test_loss = fit_random_forest(x_train=x_train, 
                                                y_train=y_train, 
                                                x_test=x_test, 
                                                y_test=y_test)
    
    ext_losses.append([train_loss, test_loss])
        
    # xgboost with SCI
    xgb_train_loss, xgb_test_loss = fit_random_forest(x_train=x_train_sci, 
                                                y_train=y_train_sci, 
                                                x_test=x_test_sci, 
                                                y_test=y_test_sci)
    
    ext_sci_losses.append([xgb_train_loss, xgb_test_loss])
    
    print(train_loss, test_loss)
    print()

Baseline train DF shape : (1976, 25)
Baseline+SCI train DF shape : (1976, 31)
0.5484403649033528 1.2167818591475341

Baseline train DF shape : (3952, 25)
Baseline+SCI train DF shape : (3952, 31)
0.6620685446377401 0.860951561381698

Baseline train DF shape : (5928, 25)
Baseline+SCI train DF shape : (5928, 31)
0.66631924571268 0.82664139735479

Baseline train DF shape : (7904, 25)
Baseline+SCI train DF shape : (7904, 31)
0.656830871300273 0.7640903356626092

Baseline train DF shape : (9880, 25)
Baseline+SCI train DF shape : (9880, 31)
0.6538564967679518 0.8304067813249559

Baseline train DF shape : (11856, 25)
Baseline+SCI train DF shape : (11856, 31)
0.6628300899343122 0.8433000393859917

Baseline train DF shape : (13832, 25)
Baseline+SCI train DF shape : (13832, 31)
0.6691598731657895 0.8209147296152023

Baseline train DF shape : (15808, 25)
Baseline+SCI train DF shape : (15808, 31)
0.6644410586553855 0.7081537022378744

Baseline train DF shape : (17784, 25)
Baseline+SCI train DF shap

In [79]:
# construct and save resulting DataFrame with XGBoost RMSEs on train and test data
ext_df = pd.DataFrame()

ext_losses = np.array(ext_losses)
ext_sci_losses = np.array(ext_sci_losses)

ext_df['xgb_losses_train'] = ext_losses[:, 0]
ext_df['xgb_losses_test'] = ext_losses[:, 1]

ext_df['xgb_sci_losses_train'] = ext_sci_losses[:, 0]
ext_df['xgb_sci_losses_test'] = ext_sci_losses[:, 1]

ext_df.to_csv('ext_df.csv', index=False)